# 辞書とAIモデルをコピーし、jarファイルを作成する

In [1]:
import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        version=l.rstrip()
        break

os.environ["VER"]=version


In [2]:
!echo $VER

1.8o


In [3]:
!rm -rf target

In [4]:
!rm -rf src/main/resources/ai/*

In [5]:
!rm -rf src/main/resources/ai_r/*

In [6]:
!rm -rf src/main/resources/dict/*

In [7]:
import glob
import shutil
for fname in glob.glob(f'../train/model/model.{version}/*.pt'):
    if "checkpoint" in fname:
        continue

    shutil.copy(fname,'src/main/resources/ai/')


In [8]:
for fname in glob.glob(f'../train/model/model.{version}/*.txt'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai/')


In [9]:
for fname in glob.glob(f'../train/model/model.{version}/*.json'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai/')

In [10]:
for fname in glob.glob(f'../train/model/model_r.{version}/*.pt'):
    if "checkpoint" in fname:
        continue

    shutil.copy(fname,'src/main/resources/ai_r/')

In [11]:
for fname in glob.glob(f'../train/model/model_r.{version}/*.txt'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai_r/')

In [12]:
for fname in glob.glob(f'../train/model/model_r.{version}/*.json'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai_r/')

In [13]:
for fname in glob.glob(f'../dict/data/data.{version}/*json'):
    shutil.copy(fname,'src/main/resources/dict/')

In [14]:
with open("kanjikana.properties",'r',encoding='utf-8') as f:
    lst = [l.rstrip() for l in f]


newlst=[]
for l in lst:
    newlst.append(l.replace(r"${project.version}",version))

with open("src/main/resources/kanjikana.properties",'w',encoding='utf-8') as f:
    for l in newlst:
        f.write(l+"\n")

In [15]:
!cat pom.xml.template |sed s/VER/$VER/ > pom.xml 


## dictpool
辞書をあらかじめクラスにロードし、シリアライズして保存しておく

In [16]:
!mvn dependency:copy-dependencies

[INFO] Scanning for projects...
[INFO] 
[INFO] ---------------< jp.go.digital.kanjikana:kanjikana_core >---------------
[INFO] Building kanjikana_core 1.8o
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-dependency-plugin:2.8:copy-dependencies (default-cli) @ kanjikana_core ---
[INFO] Copying maven-plugin-api-3.1.0.jar to /home/analysis01/src/kanjikana-model/core/target/dependency/maven-plugin-api-3.1.0.jar
[INFO] Copying aopalliance-1.0.jar to /home/analysis01/src/kanjikana-model/core/target/dependency/aopalliance-1.0.jar
[INFO] Copying jackson-databind-2.17.1.jar to /home/analysis01/src/kanjikana-model/core/target/dependency/jackson-databind-2.17.1.jar
[INFO] Copying micrometer-observation-1.12.2.jar to /home/analysis01/src/kanjikana-model/core/target/dependency/micrometer-observation-1.12.2.jar
[INFO] Copying velocity-tools-2.0.jar to /home/analysis01/src/kanjikana-model/core/target/dependency/velocity-tools-2.0.jar
[INFO] Cop

In [17]:
!mvn compile

[INFO] Scanning for projects...
[INFO] 
[INFO] ---------------< jp.go.digital.kanjikana:kanjikana_core >---------------
[INFO] Building kanjikana_core 1.8o
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:3.2.0:resources (default-resources) @ kanjikana_core ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Using 'UTF-8' encoding to copy filtered properties files.
[INFO] Copying 28 resources
[INFO] The encoding used to copy filtered properties files have not been set. This means that the same encoding will be used to copy filtered properties files as when copying other filtered resources. This might not be what you want! Run your build with --debug to see which files might be affected. Read more at https://maven.apache.org/plugins/maven-resources-plugin/examples/filtering-properties-files.html
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ kanjikana_core ---
[INFO] Chang

In [18]:
!rm -rf src/main/resources/dict_*.dat

In [19]:
!java -cp "target/classes:lib/*" jp.go.digital.kanjikana.core.engine.dict.DictPoolMain

load,/dict/statistics_1.8o.json_true
load,/dict/statistics_1.8o.json_false
load,/dict/reliable_1.8o.json_true
load,/dict/reliable_1.8o.json_false
load,/dict/unreliable_1.8o.json_true
load,/dict/unreliable_1.8o.json_false
load,/dict/tankanji_1.8o.json_true
load,/dict/tankanji_1.8o.json_false
load,/dict/itaiji_1.8o.json_true
load,/dict/itaiji_1.8o.json_false


## jarファイルにまとめる

In [20]:
!mvn clean compile assembly:single

[INFO] Scanning for projects...
[INFO] 
[INFO] ---------------< jp.go.digital.kanjikana:kanjikana_core >---------------
[INFO] Building kanjikana_core 1.8o
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ kanjikana_core ---
[INFO] Deleting /home/analysis01/src/kanjikana-model/core/target
[INFO] 
[INFO] --- maven-resources-plugin:3.2.0:resources (default-resources) @ kanjikana_core ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Using 'UTF-8' encoding to copy filtered properties files.
[INFO] Copying 28 resources
[INFO] The encoding used to copy filtered properties files have not been set. This means that the same encoding will be used to copy filtered properties files as when copying other filtered resources. This might not be what you want! Run your build with --debug to see which files might be affected. Read more at https://maven.apache.org/plugins/maven-resources-pl

In [21]:
!mv target/kanjikana_core-$VER-jar-with-dependencies.jar lib/

In [22]:
!date

2025年 10月  8日 水曜日 16:06:35 JST
